In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
import numpy as np
import torch as th
from syft import VirtualMachine
from pathlib import Path
from torchvision import datasets, transforms
from syft.core.plan.plan_builder import PLAN_BUILDER_VM, make_plan, build_plan_inputs, ROOT_CLIENT
from syft.lib.python.list import List
from matplotlib import pyplot as plt
from syft import logger
from syft.lib.torch.module import ModelExecutor
logger.remove()

In [3]:
alice = VirtualMachine()
alice_client = alice.get_root_client()

# Define Model

In [4]:
class MLP(th.nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = th.nn.Linear(8, 4)
        self.a1 = th.nn.ReLU()
        self.l2 = th.nn.Linear(4, 2)
        
    def forward(self, x):
        l1_out = self.a1(self.l1(x))
        l2_out = self.l2(l1_out)
        return l2_out

In [5]:
local_model = MLP()

In [6]:
executor = ModelExecutor(local_model)

# Define Plan

In [7]:
dummy_dl = sy.lib.python.List([
    [th.rand([4,8]), th.randint(0,2, (4,))] for _ in range(1)
])

In [8]:
@make_plan
def train(dl=dummy_dl,
          model=local_model
         ):
    remote_torch = ROOT_CLIENT.torch
    optimizer = remote_torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0)
    criterion = remote_torch.nn.CrossEntropyLoss()
    
    for xy in dl:
        x = xy[0]
        y = xy[1]
        out = executor(model,x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        
    return loss, model

/mnt/d/ubuntu/anaconda3/envs/pysyft_dev/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


# Run

In [9]:
remote_model = local_model.send(alice_client)
dummy_dl_ptr = dummy_dl.send(alice_client)
train_ptr = train.send(alice_client)

In [10]:
res_ptr = train_ptr(dl=dummy_dl_ptr, model=remote_model)

In [11]:
res = res_ptr.get()
loss = res[0]
get_model = res[1]

/mnt/d/github/PySyft/src/syft/lib/torch/uppercase_tensor.py:30: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(obj, "grad", None)


In [12]:
loss = res[0]
loss

tensor(0.7172, requires_grad=True)

In [13]:
get_model = res[1]

In [14]:
list(local_model.parameters())

[Parameter containing:
 tensor([[-0.2468, -0.3473, -0.0972,  0.1971, -0.0750,  0.2802, -0.3202,  0.3238],
         [-0.0981, -0.0860,  0.1253, -0.1503,  0.3521,  0.0640,  0.1574,  0.0462],
         [ 0.1285, -0.1593, -0.2483,  0.3037,  0.3279,  0.0436,  0.0758,  0.2627],
         [-0.2953,  0.1203, -0.2534,  0.2447, -0.0197,  0.0519, -0.3233, -0.0060]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0886,  0.0461, -0.1898,  0.1258], requires_grad=True),
 Parameter containing:
 tensor([[ 0.0588, -0.4441, -0.0490,  0.2595],
         [-0.2622,  0.3460,  0.0152,  0.3954]], requires_grad=True),
 Parameter containing:
 tensor([0.2401, 0.2451], requires_grad=True)]

In [15]:
list(get_model.parameters())

[Parameter containing:
 tensor([[-0.2481, -0.3474, -0.0990,  0.1962, -0.0776,  0.2794, -0.3203,  0.3210],
         [-0.0999, -0.0886,  0.1159, -0.1519,  0.3456,  0.0640,  0.1529,  0.0542],
         [ 0.1288, -0.1588, -0.2483,  0.3037,  0.3280,  0.0444,  0.0760,  0.2634],
         [-0.2953,  0.1203, -0.2534,  0.2447, -0.0197,  0.0519, -0.3233, -0.0060]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0920,  0.0400, -0.1894,  0.1258], requires_grad=True),
 Parameter containing:
 tensor([[ 0.0581, -0.4397, -0.0512,  0.2595],
         [-0.2615,  0.3416,  0.0175,  0.3954]], requires_grad=True),
 Parameter containing:
 tensor([0.2478, 0.2374], requires_grad=True)]